In [27]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  3 23:02:06 2021

@author: Racholsan
"""

import mysql.connector
from mysql.connector import Error
import pandas as pd
import random
import datetime
import matplotlib.pyplot as plt
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='acadejee',
                                         user='root',
                                         password= '12345')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
    
mycursor = connection.cursor()
mycursor.execute("SHOW TABLES")
cursor.fetchall()

df_o = pd.read_sql_query("select * from online_exam_scores;", connection)
df_s = pd.read_sql_query("select * from student_exam_scores;", connection)

Connected to MySQL Server version  8.0.24
You're connected to database:  ('acadejee',)


# Installing dependencies

In [28]:
import numpy as np
#import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
#import cufflinks as cf
from statsmodels import *
from plotly.subplots import make_subplots
#import chart_studio
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
#cf.go_offline()

# Changing the data types

In [29]:
df_o[['scored_marks', 'total_marks']] = df_o[['scored_marks', 'total_marks']].astype(float)
df_o['total_time'] = df_o['total_time'].astype(float)
df_o['updated_at'].astype(str).str[:10]
df_o['date'] = df_o['updated_at'].dt.strftime('%Y-%m-%d')

# Data Filtering

In [30]:
df_o  = df_o.dropna(how = 'any')
df_o  = df_o[df_o['total_time'] > 30]
df_o = df_o[df_o['total_marks'] > 99]
df_o = df_o.sort_values(by = 'scored_marks', ascending = False)

# Making new columns (nor_marks & percentile)

In [31]:
df_o['avg_marks']  = df_o.groupby(["exam_id", "sub_id", "total_marks"])['scored_marks'].transform(lambda x: x.mean())
df_o["percentile"] = (df_o.groupby(["exam_id", "sub_id", "total_marks"])['scored_marks'].rank(pct = True))*100
df_o['percentage'] = (np.divide(df_o['scored_marks'], df_o['total_marks']))*100
df_o
df_o.to_csv('acadejee.csv')

In [32]:
df_o['avg_marks'] = np.around(df_o['avg_marks'], decimals = 1)
df_o['total_time'] = np.around(df_o['total_time'], decimals = 1)
df_o['percentile'] = np.around(df_o['percentile'], decimals = 1)
df_o

,id,reg_no,exam_id,sub_id,attempt_count,scored_marks,total_marks,total_time,created_at,updated_at,date,avg_marks,percentile,percentage
701,702,ACAD-203002,14,10,501,305.0,360.0,56.7,2020-06-19 00:20:01,2020-06-19 00:20:01,2020-06-19,146.5,100.0,84.722222
674,675,ACAD-203006,14,10,504,301.0,360.0,57.1,2020-06-18 23:25:41,2020-06-18 23:25:41,2020-06-18,146.5,90.0,83.611111
1427,1431,ACAD-203002,32,10,873,299.0,360.0,79.1,2020-07-19 18:26:51,2020-07-19 18:26:51,2020-07-19,223.7,100.0,83.055556
793,794,ACAD-203003,16,10,584,277.0,360.0,87.3,2020-06-21 18:38:26,2020-06-21 18:38:26,2020-06-21,163.0,100.0,76.944444
1532,1536,ACAD-203001,34,10,925,277.0,360.0,33.3,2020-07-26 17:37:22,2020-07-26 17:37:22,2020-07-26,215.5,100.0,76.944444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,3102,ACAD-208012,84,1,1831,-7.0,100.0,65.3,2021-02-28 12:47:36,2021-02-28 12:47:36,2021-02-28,-0.8,25.0,-7.000000
2221,2225,ACAD-203006,49,7,1312,-8.0,180.0,43.1,2020-09-20 11:55:06,2020-09-20 11:55:06,2020-09-20,-5.8,75.0,-4.444444
2224,2228,ACAD-203001,49,7,1308,-9.0,180.0,35.7,2020-09-20 12:04:33,2020-09-20 12:04:33,2020-09-20,-5.8,50.0,-5.000000
2680,2684,ACAD-208014,64,2,1569,-11.0,100.0,50.2,2020-11-22 11:46:44,2020-11-22 11:46:44,2020-11-22,37.8,11.1,-11.000000


In [33]:
df_o['total_candidates'] = df_o.groupby(["exam_id", "sub_id", "total_marks"])['scored_marks'].transform(len)
df_o

,id,reg_no,exam_id,sub_id,attempt_count,scored_marks,total_marks,total_time,created_at,updated_at,date,avg_marks,percentile,percentage,total_candidates
701,702,ACAD-203002,14,10,501,305.0,360.0,56.7,2020-06-19 00:20:01,2020-06-19 00:20:01,2020-06-19,146.5,100.0,84.722222,10.0
674,675,ACAD-203006,14,10,504,301.0,360.0,57.1,2020-06-18 23:25:41,2020-06-18 23:25:41,2020-06-18,146.5,90.0,83.611111,10.0
1427,1431,ACAD-203002,32,10,873,299.0,360.0,79.1,2020-07-19 18:26:51,2020-07-19 18:26:51,2020-07-19,223.7,100.0,83.055556,9.0
793,794,ACAD-203003,16,10,584,277.0,360.0,87.3,2020-06-21 18:38:26,2020-06-21 18:38:26,2020-06-21,163.0,100.0,76.944444,6.0
1532,1536,ACAD-203001,34,10,925,277.0,360.0,33.3,2020-07-26 17:37:22,2020-07-26 17:37:22,2020-07-26,215.5,100.0,76.944444,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,3102,ACAD-208012,84,1,1831,-7.0,100.0,65.3,2021-02-28 12:47:36,2021-02-28 12:47:36,2021-02-28,-0.8,25.0,-7.000000,4.0
2221,2225,ACAD-203006,49,7,1312,-8.0,180.0,43.1,2020-09-20 11:55:06,2020-09-20 11:55:06,2020-09-20,-5.8,75.0,-4.444444,4.0
2224,2228,ACAD-203001,49,7,1308,-9.0,180.0,35.7,2020-09-20 12:04:33,2020-09-20 12:04:33,2020-09-20,-5.8,50.0,-5.000000,4.0
2680,2684,ACAD-208014,64,2,1569,-11.0,100.0,50.2,2020-11-22 11:46:44,2020-11-22 11:46:44,2020-11-22,37.8,11.1,-11.000000,9.0


# Visualization

In [34]:
#x = df_o.groupby(['reg_no']).get_group(	'ACAD-203006')
#y = x.groupby(['date', 'percentile', 'sub_id','total_marks', 'exam_id']).agg({'nor_marks' : 'mean'})


fig_n = px.scatter(df_o.query("reg_no == 'ACAD-203001'"), x="date", y="percentile",
                  color = 'sub_id', size = 'total_marks', hover_data  = {'date': False , 
                                                                         'scored_marks' : ':',
                                                                          'avg_marks' : ':',
                                                                         'total_candidates' : ':'
                                                                        },
                 template = 'ggplot2', size_max = 20, title = "Performance of candidate")

#fig_n.write_html("fig_ACAD_203001.html")
fig_n.show()

In [35]:
df_o['date'] = pd.to_datetime(df_o['date'])

In [36]:
fig = px.scatter(df_o.query("reg_no == 'ACAD-203018'"), x="date", y="percentile",
                  color = 'sub_id', size = 'total_marks', template = 'plotly_dark', 
                 custom_data = ['sub_id', 'scored_marks','total_marks', 'avg_marks', 'total_candidates', 'exam_id', 'percentile',
                               'date'],trendline = 'lowess', size_max = 15)

fig.data[1]['hovertemplate'] = "<br>".join([""])
fig.data[3]['hovertemplate'] = "<br>".join([""])
fig.data[5]['hovertemplate'] = "<br>".join([""])

fig.update_traces( hovertemplate = "<br>".join([
                                  '<br><b>Your marks</b>: <b>%{customdata[1]}<br>',
                                  'Total marks: %{customdata[2]}',
                                  'Average marks: %{customdata[3]}',
                                  'Total candidates: %{customdata[4]}',
                                  'Exam ID: %{customdata[5]}',
                                  'Percentile: %{customdata[6]}',
                                  'Date: %{customdata[7]|%d-%m-%Y}',
                                  ]),
                 selector = dict(mode = "markers"))

fig.update_layout(
    hoverlabel_align = 'right',
    xaxis_title = "Date",
    yaxis_title = "Percentile",
    title = "Performance of candidate"
   )


fig.write_html("fig_ACAD_203018.html")
fig.show()



In [37]:
idx  = np.where((df_o['reg_no'] == 'ACAD-202009') & (df_o['sub_id'] == '1'))
df_ll = df_o.iloc[idx]
df_ll

,id,reg_no,exam_id,sub_id,attempt_count,scored_marks,total_marks,total_time,created_at,updated_at,date,avg_marks,percentile,percentage,total_candidates
2774,2778,ACAD-202009,69,1,1615,87.0,100.0,54.0,2020-12-13 12:53:26,2020-12-13 12:53:26,2020-12-13,75.0,83.3,87.0,6.0
1579,1583,ACAD-202009,33,1,943,87.0,100.0,45.4,2020-07-26 22:26:33,2020-07-26 22:26:33,2020-07-26,68.4,93.8,87.0,16.0
1675,1679,ACAD-202009,36,1,992,82.0,100.0,43.0,2020-08-09 16:40:45,2020-08-09 16:40:45,2020-08-09,69.3,72.2,82.0,18.0
1768,1772,ACAD-202009,38,1,1041,77.0,100.0,52.2,2020-08-23 16:34:38,2020-08-23 16:34:38,2020-08-23,66.8,56.2,77.0,16.0
1882,1886,ACAD-202009,40,1,1095,75.0,100.0,41.4,2020-09-01 16:39:54,2020-09-01 16:39:54,2020-09-01,54.4,92.9,75.0,14.0
2561,2565,ACAD-202009,59,1,1485,61.0,100.0,44.4,2020-10-11 17:02:14,2020-10-11 17:02:14,2020-10-11,49.5,90.0,61.0,10.0
870,871,ACAD-202009,15,1,591,60.0,100.0,37.4,2020-06-21 22:30:19,2020-06-21 22:30:19,2020-06-21,47.0,81.0,60.0,21.0
2146,2150,ACAD-202009,46,1,1267,44.0,100.0,36.3,2020-09-13 16:50:31,2020-09-13 16:50:31,2020-09-13,50.6,39.5,44.0,19.0
2618,2622,ACAD-202009,62,1,1517,40.0,100.0,53.7,2020-10-18 16:59:09,2020-10-18 16:59:09,2020-10-18,31.0,70.0,40.0,10.0
1483,1487,ACAD-202009,31,1,890,39.0,100.0,42.8,2020-07-19 22:30:18,2020-07-19 22:30:18,2020-07-19,29.2,64.7,39.0,17.0


In [38]:
idx  = np.where((df_o['sub_id'] == '2') & (df_o['exam_id'] == '69'))
df_2 = df_o.iloc[idx]
df_2

,id,reg_no,exam_id,sub_id,attempt_count,scored_marks,total_marks,total_time,created_at,updated_at,date,avg_marks,percentile,percentage,total_candidates
2775,2779,ACAD-202009,69,2,1615,67.0,100.0,71.4,2020-12-13 12:53:27,2020-12-13 12:53:27,2020-12-13,44.0,93.8,67.0,8.0
2838,2842,ACAD-202001,69,2,1673,67.0,100.0,43.4,2020-12-13 19:03:20,2020-12-13 19:03:20,2020-12-13,44.0,93.8,67.0,8.0
2778,2782,ACAD-202007,69,2,1608,60.0,100.0,83.8,2020-12-13 12:53:35,2020-12-13 12:53:35,2020-12-13,44.0,75.0,60.0,8.0
2787,2791,ACAD-202008,69,2,1607,43.0,100.0,73.0,2020-12-13 12:58:32,2020-12-13 12:58:32,2020-12-13,44.0,62.5,43.0,8.0
2781,2785,ACAD-202004,69,2,1609,35.0,100.0,85.6,2020-12-13 12:57:25,2020-12-13 12:57:25,2020-12-13,44.0,50.0,35.0,8.0
2790,2794,ACAD-202016,69,2,1619,32.0,100.0,60.7,2020-12-13 13:06:52,2020-12-13 13:06:52,2020-12-13,44.0,37.5,32.0,8.0
2769,2773,ACAD-202015,69,2,1613,25.0,100.0,85.7,2020-12-13 12:42:55,2020-12-13 12:42:55,2020-12-13,44.0,25.0,25.0,8.0
2763,2767,ACAD-202018,69,2,1616,23.0,100.0,91.4,2020-12-13 12:34:58,2020-12-13 12:34:58,2020-12-13,44.0,12.5,23.0,8.0


In [39]:
arr = np.sort(np.array(df_o.query("sub_id == '1' & reg_no == 'ACAD-202012' ")['exam_id']))
arr

fig = go.Figure(layout=go.Layout(height=600, width=1000))
fig.add_trace(go.Bar(x= arr,
                y= df_o.query("sub_id == '1' & reg_no == 'ACAD-202012' ")['scored_marks'] ,
                hoverinfo = (['all']),
                name='ACAD-202012',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x= arr,
                y= df_o.query("sub_id == '1' & percentile == 100")['scored_marks'],
                hoverinfo = (['all']),
                name='Topper',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Performance of the candidate',
    xaxis_title = 'Exam ID',
    legend_orientation = 'h',
    xaxis_titlefont_size = 16,
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Marks Scored',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()
fig.write_html("fig_ACAD_202012_performance.html")